In [ ]:
%%HTML
<!-- Mejorar visualización en proyector -->
<style>
.rendered_html {font-size: 1.2em; line-height: 150%;}
div.prompt {min-width: 0ex; padding: 0px;}
.container {width:95% !important;}
</style>

In [ ]:
import numpy as np
import scipy.signal
%matplotlib notebook
from matplotlib import animation
import matplotlib.pyplot as plt

from ipywidgets import interact, FloatSlider, IntSlider, SelectionSlider, Layout, Button, Box, Output

slider_layout = Layout(width='600px', height='20px')
slider_style = {'description_width': 'initial'}
from functools import partial
FloatSlider_nice = partial(FloatSlider, style=slider_style, layout=slider_layout, continuous_update=False)
SelectionSlider_nice = partial(SelectionSlider, style=slider_style, layout=slider_layout, continuous_update=False)

# Salida enriquecida

Por defecto la última linea ejecutada en un bloque de código imprime su resultado como string

¿Y si queremos mostrar más que sólo la última linea?

¿Y si queremos mostrar datos que no son interpretables como strings?

Podemos usar las clases y funciones de "salida enriquecida" del modulo [`IPython.display`](https://ipython.readthedocs.io/en/stable/api/generated/IPython.display.html)



### Función `display`

In [ ]:
from IPython.display import display

### Objeto `Image`

El objeto `Image`

### Objeto `Audio`

Proporciona un reproductor de sonido

Las opciones de entrada son:

1. Path a un archivo (por ejemplo `wav` o `ogg`)
1. URL apuntando un archivo
1. `ndarray` de una dimensión (mono), dos dimensiones (stereo) o más


In [ ]:
from IPython.display import Audio

t, dt = np.linspace(0.0, 0.5, num=20000, retstep=True); 
data1 = np.cos(2.0*np.pi*t*220 + (100/8)*np.sin(2.0*np.pi*t*8))
data2 = scipy.signal.chirp(t, f0=5000, f1=3000, t1=t[-1], method='quadratic')

display(Audio(data1, rate=1.0/dt),
        Audio(data2, rate=1.0/dt))

###  Objeto `Code`

Muestra código fuente con sintaxis con colores

La entrada puede ser un path o un string 

In [ ]:
from IPython.display import Code

display(Code("import foo\nfoo.bar()"),
        Code("script_interesante.py"))

### Objeto `HTML`

# Jupyter widgets

- sliders/buttons
- audio
- images/video

In [ ]:
Nt, Nf = 1000, 200
time = np.linspace(0, 5, num=Nt).reshape(-1, 1)
freq = 1  + 2*np.random.rand(Nf)
phase = np.random.randn(Nf)
s = np.sin(2.0*np.pi*time*freq+phase)
next_button = Button(description="Next")
fig, ax = plt.subplots(figsize=(6, 4), tight_layout=True)

idx = 0
line, = ax.plot(time, s[:, idx])

def plot_features(idx):
    ax.set_title(str(idx))
    line.set_ydata(s[:, idx])

def on_nbutton_clicked(b):
    global idx
    if idx < Nt:
        idx += 1
    plot_features(idx)
                
next_button.on_click(on_nbutton_clicked)
next_button

In [ ]:
plt.close('all'); fig, ax = plt.subplots(figsize=(7, 3))
line = ax.plot(t, 2*x)
ax.set_xticks([0, 1/f, 2/f, 3/f]); ax.set_xticklabels(["0", r"$1/f_0$", r"$2/f_0$", r"$3/f_0$"]);
ax.set_title(r"$\cos(2\pi t f_0) + A \sin(2\pi t k f_0)$")

f = 1.51518
t = np.linspace(0, 3/f, num=500); x = np.cos(2.0*np.pi*f*t)


update = lambda k, A: line[0].set_ydata(x + A*np.sin(2.0*np.pi*t*f*k))
interact(update, k=SelectionSlider_nice(options=[1, 2, 3, 4, 5, 6]), 
         A=FloatSlider_nice(min=0.5, max=1.0, value=0.5));

In [ ]:
plt.close('all'); fig, ax = plt.subplots(2, figsize=(7, 4))
t = np.arange(-4, 4, step=1e-2)

def square_pulse(t, a=0, T=1):
    s = np.zeros(shape=t.shape)
    s[np.absolute(t-a)<T] = 0.5 + 0.5*(t[np.absolute(t-a)<T]-a)/T 
    return s
lap_pulse = lambda t, a=0, s=1 : np.exp(-np.absolute(t-a)/s)
conv_s = np.convolve(square_pulse(t), lap_pulse(t), mode='same')

def update(a = 0): 
    ax[0].cla(); ax[1].cla()
    p1, p2 = square_pulse(t, 0.1*a - 4), lap_pulse(t)
    ax[0].plot(t, p2); ax[0].plot(t, p1); 
    ax[0].fill_between(t, 0, p1*p2, alpha=0.5)
    ax[1].plot(t, conv_s[::-1]); ax[1].scatter(0.1*a -4, np.sum(p1*p2), s=100, c='k')
    
anim = animation.FuncAnimation(fig, update, frames=80, interval=100, blit=True)

In [ ]:
# Ref: https://ipython-books.github.io/117-creating-a-sound-synthesizer-in-the-notebook/
duration, sample_rate = .25, 44000.
t = np.linspace(0., duration, int(sample_rate*duration))
synth = lambda f: np.sin(2. * np.pi * f * t)
notes = 'C,C#,D,D#,E,F,F#,G,G#,A,A#,B,C'.split(',')
freqs = 440. * 2**(np.arange(3, 3 + len(notes)) / 12.)
buttons, layout_synth = [], Layout(width='40px', height='60px', border='1px solid black')
for note, freq in zip(notes, freqs):
    button = Button(description=note, layout=layout_synth)
    def on_button_clicked(f, b):
        with Output(): # suppress the audio widget output 
            display(Audio(synth(f), rate=sample_rate, autoplay=True))
    button.on_click(partial(on_button_clicked, freq))
    buttons.append(button)
Box(children=buttons)

### Para el futuro

- [Binder](https://mybinder.org/)